In [1]:
from __future__ import print_function, division
import scipy
import cv2
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import scipy
from glob import glob
import os
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import hamming_loss


os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 # maximun alloc gpu50% of MEM
config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)


Using TensorFlow backend.
/home/nvidia/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nvidia/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nvidia/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nvidia/anaconda3/lib/python3.6/site-packages/tenso

In [2]:
class DataLoader():
    def __init__(self, dataset_name, img_res=(320, 320)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        if domain=="A":
            path = glob('PatchImage/*')
        if domain=="B":
            path = glob('RealImage/*')

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('PatchImage/*')
        path_B = glob('RealImage/*')

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [3]:
class CycleGAN():
    
    def customLoss_a(self,gen_a):
        
        def lossFunction(y_true,y_pred):  
            one_a = tf.ones_like(y_true)
            zero_a = tf.zeros_like(y_true) 
            l_a = tf.where(y_true ==-1., x=zero_a, y=one_a)
            
            one_b = tf.ones_like(y_pred)
            zero_b = tf.zeros_like(y_pred) 
            l_b = tf.where(y_pred ==-1., x=zero_b, y=one_b)
            
            loss_a = K.mean(K.abs(l_a * gen_a - y_true), axis=-1)
            loss_b = K.mean(K.abs(l_b * gen_a - y_pred), axis=-1)
            
            loss_mae = K.mean(K.abs(y_pred - y_true), axis=-1)
            
            return 0.05*loss_a +0.9*loss_mae + 0.05*loss_b
        
        return lossFunction
    
    def customLoss_b(self,gen_b):
        
        def lossFunction(y_true,y_pred):  
            one = tf.ones_like(gen_b)
            zero = tf.zeros_like(gen_b) 
            l = tf.where(gen_b ==-1., x=zero, y=one)
            
            loss_a = K.mean(K.abs(l * y_true - gen_b), axis=-1)
            loss_b = K.mean(K.abs(l * y_pred - gen_b), axis=-1)
            loss_mae = K.mean(K.abs(y_pred - y_true), axis=-1)
            
            return 0.05*loss_a + 0.9*loss_mae + 0.05*loss_b
        
        return lossFunction 
    
    
    def __init__(self):
        # Input shape
        self.img_rows = 320
        self.img_cols = 320
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'test'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 16
        self.df = 32

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

        optimizer = Adam(0.0002, 0.05)

        # Build and compile the discriminators
        self.d_p = self.build_discriminator()
        self.d_r = self.build_discriminator()
        self.d_p.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])  
        self.d_r.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        patch_A = Input(shape=self.img_shape)
        real_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        gen_A = self.g_AB(patch_A)
        gen_B = self.g_BA(real_B)
        # Translate images back to original domain
        rebulid_A = self.g_BA(gen_A)
        rebulid_B = self.g_AB(gen_B)
        #Identity mapping of images
        img_A_id = self.g_BA(patch_A)
        img_B_id = self.g_AB(real_B)

        # For the combined model we will only train the generators
        self.d_r.trainable = False
        self.d_p.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_r(gen_A)
        valid_B = self.d_p(gen_B)
        
        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[patch_A, real_B],
                              outputs=[ valid_A, valid_B,
                                        rebulid_A, rebulid_B,
                                        img_A_id, img_B_id
                                      ])
        self.combined.compile(loss=['mse', 'mse',
                                    self.customLoss_a(gen_A), self.customLoss_b(gen_B),
                                    'mae', 'mae'
                                   ],
                            loss_weights=[  1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id 
                                            ],
                            optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*16)

        # Upsampling
        u1 = deconv2d(d5, d4, self.gf*4)
        u2 = deconv2d(u1, d3, self.gf*4)
        u3 = deconv2d(u2, d2, self.gf*2)
        u4 = deconv2d(u3, d1, self.gf)

        u5 = UpSampling2D(size=2)(u4)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u5)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            for batch_i, (patch_A, real_B) in enumerate(self.data_loader.load_batch(batch_size)):
                
                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                gen_A = self.g_AB.predict(patch_A)
                gen_B = self.g_BA.predict(real_B)

                # Train the discriminators (original images = real / translated = Fake)
                dr_loss_real = self.d_r.train_on_batch(real_B, valid)
                dr_loss_fake = self.d_r.train_on_batch(gen_A, fake)
                dr_loss = 0.5 * np.add(dr_loss_real, dr_loss_fake)

                dp_loss_real = self.d_p.train_on_batch(patch_A, valid)
                dp_loss_fake = self.d_p.train_on_batch(gen_B, fake)
                dp_loss = 0.5 * np.add(dp_loss_real, dp_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dr_loss, dp_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([patch_A, real_B],
                                                        [valid, valid,
                                                        patch_A, real_B,
                                                        patch_A, real_B
                                                        ])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, epochs,
                                                                            batch_i, self.data_loader.n_batches,
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
                    self.g_AB.save("gen_model/%d_%d_model.h5" % (epoch, batch_i))

    def sample_images(self, epoch, batch_i):
        r, c = 2, 3

        patch_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        real_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)

        # Demo (for GIF)
        #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        gen_A = self.g_AB.predict(patch_A)
        gen_B = self.g_BA.predict(real_B)
        # Translate back to original domain
        rebulid_A = self.g_BA.predict(gen_A)
        rebulid_B = self.g_AB.predict(gen_B)
        
        gen_imgs = np.concatenate([patch_A, gen_A, rebulid_A, real_B, gen_B, rebulid_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Generate', 'Rebuild']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("saved_img/%d_%d.png" % (epoch, batch_i))
        plt.close()


In [ ]:
if __name__ == '__main__':
    gan = CycleGAN()
    gan.train(epochs=300, batch_size=1, sample_interval=200)

/home/nvidia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:74: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/nvidia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/nvidia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/nvidia/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/300] [Batch 0/292] [D loss: 10.378310, acc:   9%] [G loss: 67.073784, adv: 23.702950, recon: 0.912731, id: 0.933302] time: 0:00:24.216964 


/home/nvidia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


[Epoch 0/300] [Batch 1/292] [D loss: 5.951986, acc:  20%] [G loss: 33.002968, adv: 7.645895, recon: 0.795202, id: 1.242858] time: 0:00:26.124370 
[Epoch 0/300] [Batch 2/292] [D loss: 11.773167, acc:  11%] [G loss: 31.958248, adv: 7.666613, recon: 0.746874, id: 1.185756] time: 0:00:26.309116 
[Epoch 0/300] [Batch 3/292] [D loss: 3.389850, acc:  36%] [G loss: 21.997864, adv: 4.151436, recon: 0.601163, id: 1.116565] time: 0:00:26.471696 
[Epoch 0/300] [Batch 4/292] [D loss: 1.124404, acc:  31%] [G loss: 16.159056, adv: 2.311414, recon: 0.494730, id: 1.117024] time: 0:00:26.639646 
[Epoch 0/300] [Batch 5/292] [D loss: 0.768715, acc:  46%] [G loss: 14.526141, adv: 2.155033, recon: 0.434683, id: 0.889967] time: 0:00:26.806413 
[Epoch 0/300] [Batch 6/292] [D loss: 0.521330, acc:  39%] [G loss: 16.762691, adv: 1.704906, recon: 0.587488, id: 1.058823] time: 0:00:26.989351 
[Epoch 0/300] [Batch 7/292] [D loss: 0.530204, acc:  40%] [G loss: 15.570714, adv: 1.801103, recon: 0.527575, id: 0.920464]

[Epoch 0/300] [Batch 57/292] [D loss: 0.448335, acc:  27%] [G loss: 6.978753, adv: 0.904620, recon: 0.235204, id: 0.234660] time: 0:00:37.105702 
[Epoch 0/300] [Batch 58/292] [D loss: 0.436639, acc:  26%] [G loss: 6.918821, adv: 1.017983, recon: 0.219776, id: 0.240632] time: 0:00:37.342106 
[Epoch 0/300] [Batch 59/292] [D loss: 0.398607, acc:  37%] [G loss: 7.018600, adv: 0.906778, recon: 0.229172, id: 0.233554] time: 0:00:37.526661 
[Epoch 0/300] [Batch 60/292] [D loss: 0.482586, acc:  27%] [G loss: 6.252982, adv: 0.988965, recon: 0.190432, id: 0.254949] time: 0:00:37.739399 
[Epoch 0/300] [Batch 61/292] [D loss: 0.453716, acc:  34%] [G loss: 6.575969, adv: 0.985309, recon: 0.203840, id: 0.228586] time: 0:00:37.952270 
[Epoch 0/300] [Batch 62/292] [D loss: 0.452093, acc:  35%] [G loss: 6.146145, adv: 1.022521, recon: 0.184667, id: 0.203943] time: 0:00:38.168059 
[Epoch 0/300] [Batch 63/292] [D loss: 0.368402, acc:  50%] [G loss: 7.072746, adv: 1.033762, recon: 0.220890, id: 0.193369] 

...

[Epoch 2/300] [Batch 99/292] [D loss: 0.188782, acc:  74%] [G loss: 5.692449, adv: 1.311680, recon: 0.143648, id: 0.100054] time: 0:02:44.984903 
[Epoch 2/300] [Batch 100/292] [D loss: 0.183196, acc:  75%] [G loss: 4.893760, adv: 1.086732, recon: 0.126537, id: 0.098970] time: 0:02:45.200192 
[Epoch 2/300] [Batch 101/292] [D loss: 0.126236, acc:  83%] [G loss: 6.036669, adv: 1.462695, recon: 0.144507, id: 0.099573] time: 0:02:45.411148 
[Epoch 2/300] [Batch 102/292] [D loss: 0.214917, acc:  72%] [G loss: 9.987576, adv: 2.892584, recon: 0.198221, id: 0.094537] time: 0:02:45.585019 
[Epoch 2/300] [Batch 103/292] [D loss: 0.283459, acc:  73%] [G loss: 6.865877, adv: 1.340274, recon: 0.196915, id: 0.102708] time: 0:02:45.780972 
[Epoch 2/300] [Batch 104/292] [D loss: 0.168987, acc:  77%] [G loss: 5.986586, adv: 0.941038, recon: 0.191353, id: 0.124159] time: 0:02:45.989525 
[Epoch 2/300] [Batch 105/292] [D loss: 0.169660, acc:  73%] [G loss: 5.029202, adv: 0.889414, recon: 0.150786, id: 0.10

[Epoch 2/300] [Batch 156/292] [D loss: 0.134054, acc:  80%] [G loss: 4.523905, adv: 0.784058, recon: 0.138265, id: 0.096546] time: 0:02:56.698135 
[Epoch 2/300] [Batch 157/292] [D loss: 0.120054, acc:  84%] [G loss: 4.442711, adv: 0.889229, recon: 0.124022, id: 0.089676] time: 0:02:56.923535 
[Epoch 2/300] [Batch 158/292] [D loss: 0.164662, acc:  72%] [G loss: 5.229732, adv: 0.939477, recon: 0.157423, id: 0.103981] time: 0:02:57.137312 
[Epoch 2/300] [Batch 159/292] [D loss: 0.123333, acc:  85%] [G loss: 5.360623, adv: 0.996691, recon: 0.156115, id: 0.102526] time: 0:02:57.335399 
[Epoch 2/300] [Batch 160/292] [D loss: 0.150350, acc:  76%] [G loss: 4.243625, adv: 0.799039, recon: 0.123102, id: 0.083665] time: 0:02:57.542422 
[Epoch 2/300] [Batch 161/292] [D loss: 0.242317, acc:  60%] [G loss: 4.549976, adv: 0.889164, recon: 0.130034, id: 0.086103] time: 0:02:57.736835 
[Epoch 2/300] [Batch 162/292] [D loss: 0.108234, acc:  86%] [G loss: 4.756956, adv: 0.864587, recon: 0.141583, id: 0.0

[Epoch 2/300] [Batch 212/292] [D loss: 0.130363, acc:  84%] [G loss: 4.633904, adv: 0.907771, recon: 0.130950, id: 0.111721] time: 0:03:08.331237 
[Epoch 2/300] [Batch 213/292] [D loss: 0.112984, acc:  90%] [G loss: 4.456904, adv: 0.876142, recon: 0.123854, id: 0.111681] time: 0:03:08.532582 
[Epoch 2/300] [Batch 214/292] [D loss: 0.133443, acc:  80%] [G loss: 4.874435, adv: 0.966591, recon: 0.135075, id: 0.143887] time: 0:03:08.746752 
[Epoch 2/300] [Batch 215/292] [D loss: 0.147203, acc:  79%] [G loss: 5.207821, adv: 0.946342, recon: 0.152680, id: 0.148629] time: 0:03:08.962707 
[Epoch 2/300] [Batch 216/292] [D loss: 0.182506, acc:  72%] [G loss: 4.809268, adv: 0.909113, recon: 0.139210, id: 0.108301] time: 0:03:09.144771 
[Epoch 2/300] [Batch 217/292] [D loss: 0.067411, acc:  93%] [G loss: 4.993320, adv: 0.934645, recon: 0.143011, id: 0.132119] time: 0:03:09.321746 
[Epoch 2/300] [Batch 218/292] [D loss: 0.141712, acc:  80%] [G loss: 6.549908, adv: 1.118944, recon: 0.202824, id: 0.1

[Epoch 2/300] [Batch 268/292] [D loss: 0.156978, acc:  81%] [G loss: 6.806736, adv: 1.205774, recon: 0.204027, id: 0.112755] time: 0:03:19.688150 
[Epoch 2/300] [Batch 269/292] [D loss: 0.354360, acc:  41%] [G loss: 6.669388, adv: 1.360474, recon: 0.183385, id: 0.116009] time: 0:03:19.874045 
[Epoch 2/300] [Batch 270/292] [D loss: 0.197717, acc:  68%] [G loss: 6.588047, adv: 1.012542, recon: 0.212729, id: 0.105189] time: 0:03:20.070064 
[Epoch 2/300] [Batch 271/292] [D loss: 0.196290, acc:  67%] [G loss: 8.605644, adv: 1.056300, recon: 0.300489, id: 0.168751] time: 0:03:20.274280 
[Epoch 2/300] [Batch 272/292] [D loss: 0.080059, acc:  96%] [G loss: 7.839763, adv: 1.064217, recon: 0.266924, id: 0.124561] time: 0:03:20.466064 
[Epoch 2/300] [Batch 273/292] [D loss: 0.098088, acc:  92%] [G loss: 7.349564, adv: 1.208187, recon: 0.230507, id: 0.113122] time: 0:03:20.685479 
[Epoch 2/300] [Batch 274/292] [D loss: 0.040904, acc:  99%] [G loss: 6.931739, adv: 0.970107, recon: 0.232638, id: 0.1

[Epoch 3/300] [Batch 34/292] [D loss: 0.122007, acc:  85%] [G loss: 4.818871, adv: 1.040717, recon: 0.122991, id: 0.092401] time: 0:03:31.481903 
[Epoch 3/300] [Batch 35/292] [D loss: 0.139630, acc:  87%] [G loss: 4.611680, adv: 0.592383, recon: 0.156746, id: 0.097439] time: 0:03:31.680985 
[Epoch 3/300] [Batch 36/292] [D loss: 0.254604, acc:  72%] [G loss: 6.351595, adv: 1.182566, recon: 0.185038, id: 0.096255] time: 0:03:31.877066 
[Epoch 3/300] [Batch 37/292] [D loss: 0.097803, acc:  90%] [G loss: 5.421885, adv: 0.887528, recon: 0.166097, id: 0.111539] time: 0:03:32.049048 
[Epoch 3/300] [Batch 38/292] [D loss: 0.292439, acc:  56%] [G loss: 4.495502, adv: 0.733008, recon: 0.139328, id: 0.092627] time: 0:03:32.257798 
[Epoch 3/300] [Batch 39/292] [D loss: 0.157023, acc:  75%] [G loss: 4.863860, adv: 0.759338, recon: 0.152584, id: 0.126937] time: 0:03:32.464306 
[Epoch 3/300] [Batch 40/292] [D loss: 0.196226, acc:  64%] [G loss: 5.383166, adv: 0.891166, recon: 0.166935, id: 0.097113] 

In [2]:
from keras.models import load_model

g_AB = load_model("gen_model/299_200_model.h5",custom_objects={'InstanceNormalization':InstanceNormalization})
file_dir = "PatchImage"
path = "gen_img/"

def imread(path):
    return scipy.misc.imread(path, mode='RGB').astype(np.float)

for root, dirs, files in os.walk(file_dir): 
    for file in files:
        patch_a = []
        file_path = os.path.join(root, file)
        img = imread(file_path)
        patch_a.append(img)
        final_img = np.array(patch_a)/127.5 - 1.
        gen_A = g_AB.predict(final_img)
        gen_pic_A = image.array_to_img((gen_A[0]+1) * 127.5, scale=False)
        gen_pic_A.save(path+file)

/home/nvidia/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/home/nvidia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
